In [4]:
from transformers import TFBertModel, BertTokenizer
# from transformers import BertTokenizer
import tensorflow as tf
import numpy as np

In [8]:
model = tf.keras.models.load_model('/home/alridho/Coding/jentara/sentimen.h5')

<tensorflow.python.saved_model.load.Loader._recreate_base_user_object.<locals>._UserObject at 0x7bce091e5430>

In [17]:
from Sastrawi.StopWordRemover.StopWordRemoverFactory import StopWordRemoverFactory
import re
import string

factory = StopWordRemoverFactory()
stopword_remover = factory.create_stop_word_remover()
stopwords = set(factory.get_stop_words())

checkpoint = "indobenchmark/indobert-base-p2"

tokenizer = BertTokenizer.from_pretrained(checkpoint)


def prep_pred(text):
    text = text.replace('\n', ' ')
    text = re.sub(f'[{re.escape(string.punctuation)}]', '', text)
    text = text.lower()
    text = re.sub(r'\s+', ' ', text).strip()
    words = text.split()
    words = [word for word in words if word not in stopwords]
    text = ' '.join(words)
    tokenized_text = tokenizer.encode_plus(
        text,
        max_length=512,
        truncation=True,
        padding='max_length',
        add_special_tokens=True,
        return_tensors='tf',
    )
    return {
        'input_ids': tf.cast(tokenized_text['input_ids'], tf.float64),
        'attention_mask': tf.cast(tokenized_text['attention_mask'], tf.float64)
    }


def print_pred(text):
    res = prep_pred(text)
    res = model.predict(res)
    res = np.argmax(res[0]) + 1
    return print(f"Review: {text}\nPred: {res} bintang")

In [18]:
text1 = "Tempatnya sangat bagus, sejuk, suasananya indah"
text2 = "Tempatnya sangat buruk, bangunan rusak, saya tidak rekomendasikan"

print_pred(text1)

AttributeError: '_UserObject' object has no attribute 'predict'

In [16]:
print_pred(text2)

1/1 [==============================] - 1s 1s/step
Review: Tempatnya sangat buruk, bangunan rusak, saya tidak rekomendasikan
Pred: 2 bintang


In [18]:
text3 = "Ancol Taman Impian mempunyai pantai yang indah yang merupakan destinasi wisata pantai dalam kota Jakarta. Pengunjung kawasan Ancol dapat menikmati suasana segar di area Pantai Lagoon, Festival, Indah, Beach Pool dan Carnaval, serta Danau Impian, sepanjang kurang lebih 6,5 km. Wisata pantai ini semakin sempurna dengan adanya promenade hampir di sepanjang pantai serta fasilitas kuliner yang lengkap, antara lain: Jimbaran Resto, Le Bridge, Starbucks, Segarra, Talaga Sampireun dan lain-lain. Hal tersebut menjadikan wisata pantai Ancol ini lebih menarik untuk dikunjungi.Rekreasi di Ancol Taman Impian semakin menyenangkan dengan kehadiran Kereta Wisata Sato-Sato. Transportasi massal ini selain bisa mengantarkan wisatawan ke berbagai destinasi seperti Dunia Fantasi, Ocean Dream Samudra, Allianz Ecopark, dan yang lainnya, juga bisa menjadi wahana rekreasi sambil menikmati pemandangan sepanjang Ancol. Jalur keretanya membentang sepanjang 3,6 kilometer, dari Gerbang Timur hingga Pantai Bende, dan dilanjut lagi hingga Ocean Dream Samudra, lalu ke Dunia Fantasi. Kereta Sato-sato akan beroperasi mulai pukul 08.00-18.00 WIB. Ada tiga rangkaian kereta yang bisa mengangkut penumpang hingga  400 orang. Wahana kereta tersebut juga ramah bagi difable atau orang berkebutuhan khusus. Di salah satu gerbongnya dapat dikhususkan bagi warga difable. Semua wisatawan tetap bisa menikmati wahana ini tanpa bayar."

In [19]:
print_pred(text3)

1/1 [==============================] - 2s 2s/step
Review: Ancol Taman Impian mempunyai pantai yang indah yang merupakan destinasi wisata pantai dalam kota Jakarta. Pengunjung kawasan Ancol dapat menikmati suasana segar di area Pantai Lagoon, Festival, Indah, Beach Pool dan Carnaval, serta Danau Impian, sepanjang kurang lebih 6,5 km. Wisata pantai ini semakin sempurna dengan adanya promenade hampir di sepanjang pantai serta fasilitas kuliner yang lengkap, antara lain: Jimbaran Resto, Le Bridge, Starbucks, Segarra, Talaga Sampireun dan lain-lain. Hal tersebut menjadikan wisata pantai Ancol ini lebih menarik untuk dikunjungi.Rekreasi di Ancol Taman Impian semakin menyenangkan dengan kehadiran Kereta Wisata Sato-Sato. Transportasi massal ini selain bisa mengantarkan wisatawan ke berbagai destinasi seperti Dunia Fantasi, Ocean Dream Samudra, Allianz Ecopark, dan yang lainnya, juga bisa menjadi wahana rekreasi sambil menikmati pemandangan sepanjang Ancol. Jalur keretanya membentang sepanjang

In [20]:
text4 = "semakin buruk, mobil ga bisa keliling didalam lagi.. alesan macet 🤣.. sumpah nyesel ke ancol lagi.. bey ancol ga akan saya rekomendasikan sebagai rekreasi untuk keluarga lagi seperti taman mini yg udah ga nyaman lagi.. kebijakan pemprov yg sangat blunder dan ngaco.. sebelum bikin kebijakan silahkan coba dulu.. panas bgt coy.. kalo ujan badai juga bakal kuyup fasilitas 0 besar."
print_pred(text4)

1/1 [==============================] - 2s 2s/step
Review: semakin buruk, mobil ga bisa keliling didalam lagi.. alesan macet 🤣.. sumpah nyesel ke ancol lagi.. bey ancol ga akan saya rekomendasikan sebagai rekreasi untuk keluarga lagi seperti taman mini yg udah ga nyaman lagi.. kebijakan pemprov yg sangat blunder dan ngaco.. sebelum bikin kebijakan silahkan coba dulu.. panas bgt coy.. kalo ujan badai juga bakal kuyup fasilitas 0 besar.
Pred: 2 bintang
